In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# Load the data
data = pd.read_csv('master1.csv')  # Update with the path to your data
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Train-test split
train_size = int(len(data) * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]

# Fit the SARIMA model
order = (1, 1, 1)  # Specify the non-seasonal (p, d, q) components
seasonal_order = (1, 1, 1, 12)  # Specify the seasonal (P, D, Q, S) components
model = SARIMAX(train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()

# Evaluate the model
predictions = result.predict(start=test.index[0], end=test.index[-1], dynamic=False)
mse = mean_squared_error(test, predictions)
print("Mean Squared Error:", mse)

# Plot the actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Training')
plt.plot(test.index, test, label='Testing')
plt.plot(predictions.index, predictions, label='Predicted', color='red')
plt.title('Actual vs. Predicted')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

# Forecasting
forecast_periods = 30  # Number of periods to forecast
forecast = result.forecast(steps=forecast_periods)

# Plot the forecast
plt.figure(figsize=(10, 6))
plt.plot(data.index, data, label='Actual')
plt.plot(forecast.index, forecast, label='Forecast', color='green')
plt.title('Forecast for Next 30 Days')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()


C:\Users\Matt\AppData\Local\Temp\ipykernel_2180\3231479730.py:8: DtypeWarning: Columns (23,24,36,49,50,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('master1.csv')  # Update with the path to your data


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).